# Toy Radix Benders by Proximal Bundle dual decomposition

In [1]:
PREVENT_ZERO = True
REG_WEIGHT = 0. #1e-4    # Regularization weight
MAX_NONZERO  = None #48*2
RELAX_LP = True

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
mdl_ref = ijomc

ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

# N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

0.873921506968


In [4]:
df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

# 0) Load changed keffs to reduce binary vars

In [5]:
import json

with open('/home/laurence/ME/data/dynamicME/kfit_changed.json') as f:
    kfit_changed = json.load(f)

changed_keffs = [kv[0] for kv in kfit_changed]

In [6]:
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel
from dynamicme.generate import copy_model

# 1) Create master, subproblems

In [7]:
from dynamicme.optimize import Variable, Constraint
import numpy as np

radix = 2.
print('Radix:',radix)
#powers = np.arange(-3,4)
powers = [-1, 0, 1]
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Get the group ID from reference model
mu_id = 'BIOMASS_Ecoli_core_w_GAM'
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    mdl_ind = cond
    mdl = copy_model(ijomc, suffix='_%s'%mdl_ind)            
    opt = Optimizer(mdl)
    gap = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)
    #----------------------------------------------------
    # Now, add min abs err
    #----------------------------------------------------
    for rxn in gap.reactions:
        rxn.objective_coefficient = 0.
    mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
    sp = Variable('sp_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sn = Variable('sn_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sp.objective_coefficient = (1.-REG_WEIGHT)/(mu_meas+1) #1.
    sn.objective_coefficient = (1.-REG_WEIGHT)/(mu_meas+1) #1.
    cons = Constraint('abs_err_%s'%mdl_ind)
    cons._constraint_sense = 'E'
    cons._bound = mu_meas
    gap.add_metabolites(cons)
    gap.add_reactions([sp,sn])
    # mu - mu_meas = sp-sn
    # mu -sp + sn = mu_meas
    # min sp + sn
    sp.add_metabolites({cons:-1.})
    sn.add_metabolites({cons:1.})
    rxn_mu = gap.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    rxn_mu.add_metabolites({cons:1.})
    #----------------------------------------------------    
    dfi = df_conds[ df_conds.cond==cond]
    var_cons_dict = {}
    for rxn_ref in crowding_ref.reactions:
        if rxn_ref.id in changed_keffs:
            crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
            var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
            rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
            cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
            a0 = rxn_p.metabolites[crowding_p]
            if var_cons_dict.has_key(rxn_ref.id):
                var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
            else:
                var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        
    opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)    
    sub = LagrangeSubmodel(gap, cond)
    sub_dict[cond] = sub
    
for group_id in var_cons_dict.keys():
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):
            yid = 'binary_%s%s%s'%(group_id,k,l)
            y   = gap.reactions.get_by_id(yid)
            ### PREFER pwr=0, digit=1
            if pwr==0 and digit==1:
                y.objective_coefficient = 0.
            else:
                y.objective_coefficient = REG_WEIGHT

master = LagrangeMaster(gap)

('Radix:', 2.0)
('Powers:', [-1, 0, 1])
('Digits:', [1.0])


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:11 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [8]:
master.add_submodels(sub_dict)

# 2) Solve

In [9]:
# master.model.Params.OutputFlag = 0
master.print_iter = 1   # submodels take much longer
master.delta_mult = 0.5   #0.1
#master.delta_min  = 1e-20 #0
master.max_iter = 100
master.gaptol = 0.02

if RELAX_LP:
    sol_master = master.solve_relaxed()
else:
    sol_master = master.optimize()

        Iter     Dual UB          LB     Best LB         gap   relgap(%)       delta     time(s)
           0        1000           0           0        1000         100         0.5    0.047785
           1       8.975      -21.52           0       8.975         100        0.25    0.066895
           2       3.805      -25.98           0       3.805         100       0.125    0.081146
           3       1.371      -19.29           0       1.371         100      0.0625    0.094156
           4      0.8903      -21.95           0      0.8903         100     0.03125    0.107572
           5      0.4313      -19.14           0      0.4313         100     0.01562    0.119959
           6      0.2899       -18.4           0      0.2899         100    0.007812    0.134509
           7       0.239      -21.67           0       0.239         100    0.003906    0.146491
           8      0.1971      -26.25           0      0.1971         100    0.001953    0.159502
           9      0.1505      

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition.py:2084 UserWarning: Solution available but Master solver status=suboptimal (13).


In [10]:
master.uopt

array([ 3.53216733e-05, -1.76836747e-04, -1.21255526e-04,  1.04586128e-04,
        2.45469149e-04, -7.09252789e-05, -3.36261251e-05,  5.03928277e-08,
        4.14554483e-04,  1.69553461e-04,  4.54716169e-05, -2.17932577e-04,
       -1.71482366e-05, -1.30901162e-04, -9.77868763e-06, -1.22939339e-04,
       -7.21526881e-05,  8.06901664e-06, -1.97406326e-05,  1.74375904e-04,
        4.05020500e-05,  6.24892824e-05,  3.11053482e-05,  2.38826696e-04,
       -1.94411845e-04, -1.36113333e-04,  3.79983237e-04, -3.46477979e-05,
       -1.51783214e-04, -1.92013503e-04,  2.89324273e-04,  1.65971616e-04,
        1.42466957e-04, -3.85885409e-05, -9.10925411e-05,  1.57467194e-05,
       -7.34677288e-05, -2.04433596e-05,  1.04559865e-05, -6.86570122e-05,
       -3.73829254e-05, -3.02260742e-05, -2.19832784e-04, -1.53069188e-04,
        8.32677983e-07,  3.22147960e-04,  8.56083930e-05,  1.97179093e-05,
       -3.76131586e-04,  1.93683814e-04,  2.75413628e-04, -2.65911765e-04,
        5.90878290e-05, -

In [11]:
for sub_ind,sub in iteritems(sub_dict):
    print('-'*40)
    print('%s'%sub_ind)
    print('Obj fun=%s'%sub.model.getObjective())
    sub.optimize()
    print('ObjVal=%s'%sub.model.ObjVal)
    sp = sub.x_dict['sp_%s'%sub_ind]
    sn = sub.x_dict['sn_%s'%sub_ind]
    print('sp+sn=%s. sp=%s, sn=%s' % (sp+sn, sp, sn))

----------------------------------------
acetate
Obj fun=<gurobi.LinExpr: 3.53216732947e-05 binary_G6PDH2r00 + -0.000176836747187 binary_G6PDH2r01 + -0.000121255525755 binary_G6PDH2r02 + 0.000104586128373 binary_AKGDH00 + 0.000245469149149 binary_AKGDH01 + -7.09252789193e-05 binary_AKGDH02 + -3.36261250595e-05 binary_GND00 + 5.03928276885e-08 binary_GND01 + 0.000414554482745 binary_GND02 + 0.000169553460751 binary_ACKr00 + 4.54716168861e-05 binary_ACKr01 + -0.000217932577129 binary_ACKr02 + -1.71482365658e-05 binary_SUCDi00 + -0.000130901161583 binary_SUCDi01 + -9.77868762675e-06 binary_SUCDi02 + -0.000122939339121 binary_MDH00 + -7.21526880625e-05 binary_MDH01 + 8.06901664419e-06 binary_MDH02 + -1.97406326379e-05 binary_GLUDy00 + 0.000174375903953 binary_GLUDy01 + 4.05020499556e-05 binary_GLUDy02 + 6.24892824135e-05 binary_PGL00 + 3.1105348171e-05 binary_PGL01 + 0.000238826695636 binary_PGL02 + -0.000194411844745 binary_PGI00 + -0.000136113332701 binary_PGI01 + 0.000379983236712 binar

# 3) Validate solution

In [12]:
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
#             if abs(y)>1e-10:
#                 print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))            
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
# print('kfit_changed:',kfit_changed)

#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:    
    ijofit = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
    crowding = ijofit.metabolites.get_by_id('crowding')
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit}, combine=False)
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))

Changed keffs: 21/22
Cond=glucose. mu_meas=0.74. mu_sim=0.873922 (unfit=0.873922, error=18.1%). Error=18.1% (1.11e-10% change)
	EX_glc__D_e uptake=-10
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=0
Cond=acetate. mu_meas=0.256. mu_sim=0.389313 (unfit=0.389313, error=52.1%). Error=52.1% (1.94e-10% change)
	EX_glc__D_e uptake=0
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=-20


import json

with open('/home/laurence/ME/data/dynamicME/kfit_changed.json','w') as f:
    json.dump(kfit_changed,f)

In [13]:
0.382993-0.256

0.12699299999999997